# FOREST COVERTYPE DATA

## Importaciones

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
print('Setup complete')

Setup complete


## Creación del dataset

In [36]:
df_bosque = pd.read_csv('data/covtype.data', header=None)

In [37]:

# Renombrar las columnas
nombres_columnas = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']
wilderness_columns = []
soil_columns = []
for i in range(1, 5):
    wilderness_columns.append('Wilderness_Area_' + str(i))
for i in range(1, 41):
    soil_columns.append('Soil_Type_' + str(i))
nombres_columnas += wilderness_columns
nombres_columnas += soil_columns
nombres_columnas += ['Cover_Type']
df_bosque.columns = nombres_columnas
df_bosque.head()


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Soil_Type_40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


**JUNTAR LAS COLUMNAS ÁREAS SILVESTRES Y TIPOS DE SUELO**

En estas columnas se encuentra en código binario, siendo 0 la presencia y 1 si hay. Por tanto, sumaremos los unos y juntarlos en una columna para saber cuántos tipos hay.

In [40]:
df_bosque['Wilderness_Area'] = df_bosque[wilderness_columns].max(axis=1)
df_bosque.drop(wilderness_columns, axis=1, inplace=True)
df_bosque['Soil_Type'] = df_bosque[soil_columns].max(axis=1)
df_bosque.drop(soil_columns, axis=1, inplace=True)
df_bosque.head()


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type,Wilderness_Area,Soil_Type
0,2596,51,3,258,0,510,221,232,148,6279,5,1,1
1,2590,56,2,212,-6,390,220,235,151,6225,5,1,1
2,2804,139,9,268,65,3180,234,238,135,6121,2,1,1
3,2785,155,18,242,118,3090,238,238,122,6211,2,1,1
4,2595,45,2,153,-1,391,220,234,150,6172,5,1,1


## Paso 1

Para conseguir un dataset con una dimensión reducidad, aplica la técnica de Selección de variables basada en árbol de decisión mediante las importancias de cada variable (Decision Trees Importances):

Filtra el tablón para quedarnos solamente con las variables que aglutinan hasta el 95% de la información que se requiere para estimar la variable objetivo.
random_state=100

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel

# Separar las variables predictoras y la variable objetivo
X = df_bosque.drop('Cover_Type', axis=1)
y = df_bosque['Cover_Type']

# Ajustar un modelo de árbol de decisión
clf = DecisionTreeClassifier(random_state=100)
clf.fit(X, y)

# Seleccionar las variables importantes
sfm = SelectFromModel(clf, threshold='0.95*mean')
X_new = sfm.fit_transform(X, y)

# Obtener los nombres de las variables seleccionadas
mask = sfm.get_support()
selected_features = X.columns[mask]
print(selected_features)

Index(['Elevation', 'Aspect', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points'],
      dtype='object')
